# Futures Market Data Analysis – ICE Interview

This notebook presents an analysis of simulated futures market data for the Intercontinental Exchange. It includes customer activity, rebate programs, and product-level metrics, focusing on revenue performance over time and across markets. The main KPIs evaluated include:

- Average Daily Volume (ADV)
- Gross and Net Revenue per Contract (RPC)
- Customer profitability by volume, fees, and rebate impact

## Data Preparation

The following datasets were loaded and merged:

- `Order Book.csv`: Daily trading activity (Lots, Fees)
- `Customers.csv`: Company IDs and Locations
- `Products.csv`: Commodity codes grouped by product type
- `Rebates.csv`: Monthly rebate values (discounts on fees)

We merged these datasets on Company ID, Exchange, and Commodity Code to form a comprehensive transaction-level dataset.

In [3]:
import pandas as pd
import altair as alt

# Load data
orders = pd.read_csv("Order Book.csv", index_col=0)
customers = pd.read_csv("Customers.csv", index_col=0)
products = pd.read_csv("Products.csv", index_col=0)
rebates = pd.read_csv("Rebates.csv", index_col=0)

In [4]:
# Merge and prepare
orders['Date'] = pd.to_datetime(orders['Date'])
rebates['Month'] = pd.to_datetime(rebates['Month'])
orders['Month'] = orders['Date'].dt.to_period('M').dt.to_timestamp()

df = orders.merge(customers, on="Company ID", how="left")
df = df.merge(products, on=["Exchange", "Commodity Code"], how="left")
df = df.merge(rebates, on=["Company ID", "Exchange", "Commodity Code", "Month"], how="left")

## Net Fees and RPC Metrics

We calculate Net Fees as:
- Net Fees = Gross Fees + Rebates (Note: Rebates are negative values)

We also compute Gross and Net Revenue per Contract (RPC):

- Gross RPC = Gross Fees / Lots
- Net RPC = Net Fees / Lots

In [6]:
df['Rebates'] = df['Rebates'].fillna(0)
df['Net Fees'] = df['Fees'] + df['Rebates']  # rebates are negative

In [7]:
#df.head()

In [8]:
# Calculate RPC
rpc = df.groupby(['Month'])[['Group','Fees', 'Net Fees', 'Lots']].sum()
rpc['Gross RPC'] = rpc['Fees'] / rpc['Lots']
rpc['Net RPC'] = rpc['Net Fees'] / rpc['Lots']
rpc['Gross RPC MoM Change'] = rpc['Gross RPC'].pct_change()
rpc['Net RPC MoM Change'] = rpc['Net RPC'].pct_change()

In [72]:
rpc

,Group,Fees,Net Fees,Lots,Gross RPC,Net RPC,Gross RPC MoM Change,Net RPC MoM Change
Month,,,,,,,,
2020-01-01,Equity DerivativesEquity DerivativesEquity Der...,1.282540e+08,-40463776.61,56305365.0,2.277829,-0.718649,NaN,NaN
2020-02-01,Equity DerivativesEquity DerivativesEquity Der...,1.283696e+08,-25165035.31,56825875.0,2.259000,-0.442845,-0.008266,-0.383781
2020-03-01,Equity DerivativesEquity DerivativesEquity Der...,1.641557e+08,-71023892.13,72633831.0,2.260044,-0.977835,0.000462,1.208076
2020-04-01,Equity DerivativesEquity DerivativesEquity Der...,1.133708e+08,-34455445.36,50092364.0,2.263235,-0.687838,0.001412,-0.296570
2020-05-01,Equity DerivativesEquity DerivativesEquity Der...,1.098565e+08,-26616940.69,48246392.0,2.276989,-0.551688,0.006077,-0.197940
2020-06-01,Equity DerivativesEquity DerivativesEquity Der...,1.357646e+08,-55421660.13,60029817.0,2.261620,-0.923236,-0.006750,0.673475
2020-07-01,Equity DerivativesEquity DerivativesEquity Der...,1.205533e+08,-48116707.71,53401029.0,2.257508,-0.901045,-0.001818,-0.024036
2020-08-01,Equity DerivativesEquity DerivativesEquity Der...,1.162700e+08,-38296902.97,51541881.0,2.255835,-0.743025,-0.000741,-0.175374
2020-09-01,Equity DerivativesEquity DerivativesEquity Der...,1.430957e+08,-48265736.14,62565053.0,2.287150,-0.771449,0.013882,0.038254


## Average Daily Volume (ADV)

ADV is calculated as total monthly Lots divided by the number of trading days in that month. It helps normalize activity and compare volume seasonality across time.

In [11]:
# Calculate ADV
adv = df.groupby('Month').agg(
    Monthly_Lots=('Lots', 'sum'),
    Trading_Days=('Date', 'nunique')
)
adv['ADV'] = adv['Monthly_Lots'] / adv['Trading_Days']

In [12]:
#adv.head()

## ADV Chart (Monthly)
- This chart displays monthly fluctuations in total average daily trading volume across all products.
- Peaks and dips reflect cyclical trading patterns, with noticeable surges around March and mid-year months.
- These trends can help identify seasonal shifts in market activity and support planning for liquidity management and risk exposure.

In [14]:
# Reset index so 'Month' is a column
adv_reset = adv.reset_index()

# Create line chart
adv_line = alt.Chart(adv_reset).mark_line().encode(
    x=alt.X('Month:T', title='Month'),
    y=alt.Y('ADV:Q', title='Average Daily Volume (ADV)'),
    tooltip=['Month:T', 'ADV:Q']
).properties(
    title='Monthly Average Daily Volume (ADV)',
    width=600,
    height=400
).interactive()

adv_line

alt.Chart(...)

## Total Volume by Product Group (ADV Proxy)

This chart displays total traded volume (Lots) by product group. While not a strict ADV calculation, it serves as a strong proxy for overall market activity.

We can see that the Oil group dominates trading activity, followed by Softs and Equity Derivatives. FX shows the lowest relative volume.

This view helps identify which markets are most actively traded and may drive revenue or require targeted attention in product management.

In [16]:
group_adv = df.groupby('Group').agg(Total_Lots=('Lots', 'sum')).reset_index()

alt.Chart(group_adv).mark_bar().encode(
    x=alt.X('Group:N', sort='-y'),
    y='Total_Lots:Q',
    tooltip=['Group', 'Total_Lots']
).properties(
    title='Total Volume (ADV Proxy) by Product Group'
)

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

#### This code creates a quarterly summary of volume, revenue, and RPC by product group.

In [18]:
# Add Quarter and Year columns
df['Quarter'] = df['Date'].dt.to_period('Q').dt.to_timestamp()
df['Year'] = df['Date'].dt.year

# Group by 'Group' and 'Quarter' to create the Quarterly Market Summary
quarterly_summary = df.groupby(['Group', 'Quarter']).agg(
    Lots=('Lots', 'sum'),
    Fees=('Fees', 'sum'),
    Net_Fees=('Net Fees', 'sum'),
    Trading_Days=('Date', 'nunique')
).reset_index()

# Calculate metrics
quarterly_summary['Gross RPC'] = quarterly_summary['Fees'] / quarterly_summary['Lots']
quarterly_summary['Net RPC'] = quarterly_summary['Net_Fees'] / quarterly_summary['Lots']
quarterly_summary['ADV'] = quarterly_summary['Lots'] / quarterly_summary['Trading_Days']

#quarterly_summary.head()

## Quarterly ADV by Product Group

This line chart tracks average daily trading volume by product group over time, aggregated quarterly. Oil remains the most active market, with strong recurring volume spikes — suggesting seasonality or cyclical demand patterns.

Softs show the same cyclical nature as Oil but not as intensely.

FX and Equity Derivatives show steady volume, while maintaining relatively lower activity levels. These trends provide insight into market liquidity and may guide product strategy or client targeting.

In [20]:
alt.Chart(quarterly_summary).mark_line(point=True).encode(
    x='Quarter:T',
    y='ADV:Q',
    color='Group:N',
    tooltip=['Group', 'Quarter:T', 'ADV']
).properties(
    title='Quarterly ADV by Market Group',
    width=700,
    height=400
).interactive()

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Gross and Net RPC Over Time

The following chart shows monthly trends in revenue per contract:

- Gross RPC remains stable, while Net RPC fluctuates more due to varying rebate impact.
- The three largest drops are March 1st every year and then immediately spiking over the next couple months, showing the seasonal behaviour in rebate and fee structures. It is possible there are annual rebate resets.

In [22]:
rpc_reset = rpc.reset_index()

gross_rpc_line = alt.Chart(rpc_reset).mark_line().encode(
    x=alt.X('Month:T', title='Month'),
    y=alt.Y('Gross RPC:Q', title='Gross Revenue per Contract'),
    tooltip=['Month:T', 'Gross RPC:Q']
).properties(
    title='Gross RPC Over Time',
    width=450,
    height=400
).interactive()

In [23]:
rpc_reset = rpc.reset_index()

net_rpc_line = alt.Chart(rpc_reset).mark_line().encode(
    x=alt.X('Month:T', title='Month'),
    y=alt.Y('Net RPC:Q', title='Net Revenue per Contract'),
    tooltip=['Month:T', 'Net RPC:Q']
).properties(
    title='Net RPC Over Time',
    width=450,
    height=400
).interactive()


gross_rpc_line | net_rpc_line

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

## Gross vs. Net RPC by Product Group

These side-by-side charts compare gross and net revenue per contract (RPC) over time for each product group. While gross RPC appears stable across groups, the net RPC chart reveals significant revenue loss due to rebates.

Softs maintain the highest gross RPC, but their net RPC shows meaningful erosion. FX stands out as consistently unprofitable on a per-trade basis after rebates, raising questions about rebate strategy or contract structure.

This comparison illustrates how rebates materially impact product-level profitability, which isn't visible from gross figures alone.

In [25]:
quarter_gross_rpc = alt.Chart(quarterly_summary).mark_line(point=True).encode(
    x=alt.X('Quarter:T', title='Quarter'),
    y=alt.Y('Gross RPC:Q', title='Gross Revenue per Contract'),
    color=alt.Color('Group:N', title='Market Group'),
    tooltip=['Group', 'Quarter:T', 'Gross RPC:Q']
).properties(
    title='Quarterly Gross RPC by Market Group',
    width=450,
    height=400
).interactive()

In [26]:
quarter_net_rpc = alt.Chart(quarterly_summary).mark_line(point=True).encode(
    x=alt.X('Quarter:T', title='Quarter'),
    y=alt.Y('Net RPC:Q', title='Net Revenue per Contract (after Rebates)'),
    color=alt.Color('Group:N', title='Market Group'),
    tooltip=['Group', 'Quarter:T', 'Net RPC:Q']
).properties(
    title='Quarterly Net RPC by Market Group',
    width=450,
    height=400
).interactive()

quarter_gross_rpc | quarter_net_rpc

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

## Top 10 Customers by Net RPC

Net RPC measures profitability per contract. These customers yield the highest average revenue per trade, making them strategically valuable despite their total volume.

In [28]:
rpc_by_customer = df.groupby('Company Name').agg(
    Total_Lots=('Lots', 'sum'),
    Net_Fees=('Net Fees', 'sum')
).reset_index()

rpc_by_customer['Net_RPC'] = rpc_by_customer['Net_Fees'] / rpc_by_customer['Total_Lots']
top_net_rpc_customers = rpc_by_customer.nlargest(10, 'Net_RPC')

alt.Chart(top_net_rpc_customers).mark_bar().encode(
    x=alt.X('Net_RPC:Q', title='Net Revenue per Contract (Net RPC)'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Net_RPC']
).properties(
    title='Top 10 Customers by Net RPC',
    width=400,
    height=300
)

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Bottom 10 Customers by Net RPC

These customers lose the exchange money on a per-contract basis. They may be over-rebated or trading in high-cost, low-fee instruments.

In [30]:
# Get bottom 10 customers by Net RPC
bottom_net_rpc_customers = rpc_by_customer.nsmallest(10, 'Net_RPC')

# Bar chart
alt.Chart(bottom_net_rpc_customers).mark_bar().encode(
    x=alt.X('Net_RPC:Q', title='Net Revenue per Contract (Net RPC)'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Net_RPC']
).properties(
    title='Bottom 10 Customers by Net RPC',
    width=400,
    height=300
)

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Top 10 Customers by Net Fees

This bar chart shows which customers contributed the most in total net revenue after rebates. These are your highest grossing clients in absolute dollar terms.

In [32]:
top_customers = df.groupby('Company Name').agg(Net_Fees=('Net Fees', 'sum')).nlargest(10, 'Net_Fees').reset_index()

alt.Chart(top_customers).mark_bar().encode(
    x=alt.X('Net_Fees:Q'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Net_Fees']
).properties(
    title='Top 10 Customers by Net Fees'
)

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Bottom 10 Customers by Net Fees

This chart highlights the customers who generated the lowest total net revenue after rebates — in many cases, incurring negative net fees. These clients may be heavily subsidized through rebate programs or have trading profiles that result in high costs relative to their gross fees.

In [34]:
# bottom 10 customers by Net Fees
bottom_customers = (
    df.groupby('Company Name')
    .agg(Net_Fees=('Net Fees', 'sum'))
    .nsmallest(10, 'Net_Fees')  # <-- smallest instead of largest
    .reset_index()
)

# Altair bar chart
alt.Chart(bottom_customers).mark_bar().encode(
    x=alt.X('Net_Fees:Q', title='Net Fees'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Net_Fees']
).properties(
    title='Bottom 10 Customers by Net Fees'
)

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Top 5 Customers by Market (Volume & Net Fees)

The following code and charts break down top-performing customers across four product groups: Oil, FX, Softs, and Equity Derivatives. Each group shows the top 5 by volume and by net fees for comparison.

In [36]:
# Filter for "Oil" group only
oil_data = df[df['Group'] == 'Oil']

# Aggregate by customer
oil_summary = (
    oil_data.groupby('Company Name')
    .agg(Total_Lots=('Lots', 'sum'), Total_Net_Fees=('Net Fees', 'sum'))
    .reset_index()
)

# Get top 5 by each metric
top_lots = oil_summary.nlargest(5, 'Total_Lots')
top_fees = oil_summary.nlargest(5, 'Total_Net_Fees')

# Bar chart for volume
lots_chart = alt.Chart(top_lots).mark_bar().encode(
    x=alt.X('Total_Lots:Q', title='Total Volume (Lots)'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Lots']
).properties(
    title='Top 5 Oil Customers by Volume',
    width=300,
    height=200
)

# Bar chart for net fees
fees_chart = alt.Chart(top_fees).mark_bar().encode(
    x=alt.X('Total_Net_Fees:Q', title='Total Net Fees'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Net_Fees']
).properties(
    title='Top 5 Oil Customers by Net Fees',
    width=300,
    height=200
)

In [37]:
# Filter for "Equity Derivatives" group only
eq_data = df[df['Group'] == 'Equity Derivatives']

# Aggregate by customer
eq_summary = (
    eq_data.groupby('Company Name')
    .agg(Total_Lots=('Lots', 'sum'), Total_Net_Fees=('Net Fees', 'sum'))
    .reset_index()
)

# Get top 5 by each metric
top_lots_eq = eq_summary.nlargest(5, 'Total_Lots')
top_fees_eq = eq_summary.nlargest(5, 'Total_Net_Fees')

# Bar chart for volume
lots_chart_eq = alt.Chart(top_lots_eq).mark_bar().encode(
    x=alt.X('Total_Lots:Q', title='Total Volume (Lots)'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Lots']
).properties(
    title='Top 5 Equity Derivatives Customers by Volume',
    width=300,
    height=200
)

# Bar chart for net fees
fees_chart_eq = alt.Chart(top_fees_eq).mark_bar().encode(
    x=alt.X('Total_Net_Fees:Q', title='Total Net Fees'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Net_Fees']
).properties(
    title='Top 5 Equity Derivatives Customers by Net Fees',
    width=300,
    height=200
)

In [38]:
# Filter for "Softs" group only
softs_data = df[df['Group'] == 'Softs']

# Aggregate by customer
softs_summary = (
    softs_data.groupby('Company Name')
    .agg(Total_Lots=('Lots', 'sum'), Total_Net_Fees=('Net Fees', 'sum'))
    .reset_index()
)

# Get top 5 by each metric
top_lots_softs = softs_summary.nlargest(5, 'Total_Lots')
top_fees_softs = softs_summary.nlargest(5, 'Total_Net_Fees')

# Bar chart for volume
lots_chart_softs = alt.Chart(top_lots_softs).mark_bar().encode(
    x=alt.X('Total_Lots:Q', title='Total Volume (Lots)'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Lots']
).properties(
    title='Top 5 Softs Customers by Volume',
    width=300,
    height=200
)

# Bar chart for net fees
fees_chart_softs = alt.Chart(top_fees_softs).mark_bar().encode(
    x=alt.X('Total_Net_Fees:Q', title='Total Net Fees'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Net_Fees']
).properties(
    title='Top 5 Softs Customers by Net Fees',
    width=300,
    height=200
)

In [39]:
# Filter for "FX" group only
fx_data = df[df['Group'] == 'FX']

# Aggregate by customer
fx_summary = (
    fx_data.groupby('Company Name')
    .agg(Total_Lots=('Lots', 'sum'), Total_Net_Fees=('Net Fees', 'sum'))
    .reset_index()
)

# Get top 5 by each metric
top_lots_fx = fx_summary.nlargest(5, 'Total_Lots')
top_fees_fx = fx_summary.nlargest(5, 'Total_Net_Fees')

# Bar chart for volume
lots_chart_fx = alt.Chart(top_lots_fx).mark_bar().encode(
    x=alt.X('Total_Lots:Q', title='Total Volume (Lots)'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Lots']
).properties(
    title='Top 5 FX Customers by Volume',
    width=300,
    height=200
)

# Bar chart for net fees
fees_chart_fx = alt.Chart(top_fees_fx).mark_bar().encode(
    x=alt.X('Total_Net_Fees:Q', title='Total Net Fees'),
    y=alt.Y('Company Name:N', sort='-x'),
    tooltip=['Company Name', 'Total_Net_Fees']
).properties(
    title='Top 5 FX Customers by Net Fees',
    width=300,
    height=200
)

In [40]:
# Final dashboard layout
(lots_chart | fees_chart) & \
(lots_chart_eq | fees_chart_eq) & \
(lots_chart_softs | fees_chart_softs) & \
(lots_chart_fx | fees_chart_fx)

/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Applications/anaconda3/lib/pyt

alt.VConcatChart(...)

### Top Performing Commodities by Volume and Profitability

These side-by-side charts identify the top 10 commodities based on two key metrics:

- **Average Daily Volume (ADV):** Reflects how actively a commodity is traded, serving as a proxy for market interest and liquidity.
- **Average Net Revenue per Contract (Net RPC):** Indicates how much net revenue each contract generates after rebates, highlighting product profitability.

This comparison helps distinguish between high-volume products and those that yield the highest returns per contract — a key insight when balancing market share and margin.

In [42]:
comm_summary = df.groupby(['Name', 'Month']).agg(
    Total_Lots=('Lots', 'sum'),
    Total_Net_Fees=('Net Fees', 'sum'),
    Trading_Days=('Date', 'nunique')
).reset_index()

comm_summary['ADV'] = comm_summary['Total_Lots'] / comm_summary['Trading_Days']
comm_summary['Net_RPC'] = comm_summary['Total_Net_Fees'] / comm_summary['Total_Lots']

overall = comm_summary.groupby('Name').agg(
    Avg_ADV=('ADV', 'mean'),
    Avg_Net_RPC=('Net_RPC', 'mean')
).reset_index()


In [43]:
# Get top 10 by ADV
top_adv = overall.sort_values(by='Avg_ADV', ascending=False).head(10)

adv_chart = alt.Chart(top_adv).mark_bar().encode(
    x=alt.X('Avg_ADV:Q', title='Average Daily Volume (ADV)'),
    y=alt.Y('Name:N', sort='-x', title='Commodity'),
    tooltip=['Name', 'Avg_ADV']
).properties(
    title='Top 10 Commodities by ADV',
    width=300,
    height=300
)

# Get top 10 by Net RPC
top_rpc = overall.sort_values(by='Avg_Net_RPC', ascending=False).head(10)

rpc_chart = alt.Chart(top_rpc).mark_bar().encode(
    x=alt.X('Avg_Net_RPC:Q', title='Average Net Revenue per Contract (Net RPC)'),
    y=alt.Y('Name:N', sort='-x', title='Commodity'),
    tooltip=['Name', 'Avg_Net_RPC']
).properties(
    title='Top 10 Commodities by Net RPC',
    width=300,
    height=300
)

# Display side by side
adv_chart | rpc_chart


/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Applications/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)